---
title: Chapter 10 - Introduction to ANN
format: 
    html: 
        anchor-sections: true
        smooth-scroll: true
        code-line-numbers: true
        code-links:
            text: GitHub
            icon: github
            href: "https://github.com/Mark-Gallacher/handson-ml2"
        link-external-icon: true
        link-external-newwindow: true
theme: 
    light: flatly
    dark: darkly
toc: true
toc-depth: 2
toc-title: "Overview"
number-sections: true
number-depth: 2
---


# Basic Perceptron

## How does a Perceptron Model work?

The inputs ($X$) are assigned weights ($W$) and passed to the core unit, which is usually a *threshold logic unit* (TLU) or 
a *linear threshold unit* (LTU). In the case of a TLU, a step function is applied to generate a binary output (1 or 0). 
This is usually a *Heaviside step function*, where every non-negative value is assigned the value 1, otherwise it is 0 (@eq-heaviside).


The general formula to get the output of the Perceptron is given in @eq-perceptron. $\phi$ represents the activation function, 
which in our case is the step function but there are some alternatives, such as the Hyperbolic Tangent Function (*tanh*), Rectified Linear
Unit Function (ReLU). Finally, a bias term ($b$) is added on to every layer, apart the output layer, which usually is simply positive one.


$$
\begin{equation}
heaviside(z) =  
    \begin{cases}
         0 & \text{if }z < 0, \\ 
         1 & \text{if }z \ge 0.
    \end{cases}
\end{equation}
$$ {#eq-heaviside} 

$$ 
h_{W,b}(X) = \phi(XW + b)
$$ {#eq-perceptron}


We can play around with the Perceptron thanks to `Perceptron` in **Scikit-Learn**.


In [ ]:
from sklearn.datasets import load_iris
from sklearn.linear_model import Perceptron
import numpy as np

iris = load_iris()

x = iris.data[:, (2, 3)]
y = (iris.target == 0).astype(np.int32)

In [ ]:
per_clf = Perceptron()
per_clf.fit(x, y)

y_pred = per_clf.predict([[2, 0.5]])
print(y_pred)

## The Main Issue with Perceptrons

There's nothing really new here if you have come across artificial neural networks, but this form is too simple to be useful. One issue is that is does not 
pass the XOR test, the exclusive-OR test. Notice how even when we pass the exact same data to the `fit()` and `predict()` methods, it is unable to correctly
label the samples. This is essentially because the model is still linear, in the same sense logistic regression models are linear. Whereas the XOR problem 
requires a non-linear boundary to capture both the (0, 0) and (1, 1). 


In [ ]:
per_clf = Perceptron()

x = np.array([
    0, 0,
    1, 0,
    0, 1,
    1, 0
]).reshape(4, 2)

y = np.array([0, 1, 1, 0]).reshape(4, ) 

per_clf.fit(x, y)

y_pred = per_clf.predict(x)
print(y_pred)

So, how do solve this? One solution would be to use multiple layers of perceptrons, where the data is passed to an input layer, 
but their output is passed to a *hidden* layer. In this model, we have a fully connected network, where every unit in one layer is connected
to every node in the next layer. These types of models are much more common and useful, as they go by the name of *Multilayer Perceptrons*.

# Multilayer Perceptrons

Multilayer Perceptrons (MLP) are made up of three layers, input, hidden and output.

    - input: where the data enters the network
    - hidden: one or more layers with TLU nodes
    - output: last layer of TLU

Given that a layer is influenced by the previous layer, and each layer has a set of tuning parameters,
training a network becomes more challenging. The key breakthrough was the discovery or invention of the
backpropagation, which employs gradient descent to find the optimal update that can minimise error.

## Backpropagation

The overview of the algorithm is:

- It passes a set of data, a mini-batch, through the network until all samples are used up.
    - Each pass of the complete training set is called an *Epoch*
    - Meaning each epoch is made of multiple mini-batches.

- For each mini-batch, a *forward pass* is completed, this is basically allowing the network to
    generate an output for the supplied data. The key difference though is that each intermediate
    result (output of each hidden layer) is stored for the next step.

- Next, after we have the model predictions, we obtain the error using a loss function[^1].

- Now, we work out how much each connection to the output added to the error. 

- Then we repeat this process, starting at the output layer until we reach the input layer
    - Essentially, we want to know how much each connection contributes to the error in the next layer.

- Once we have completed our backwards pass, we have the gradients we need to correct our whole network

- So we simply compute the gradient descent uses these gradients.


## Activation Functions

If we remember the perceptrons typically used a step function as their activation function, we run into a problem.
A step function is basically two horizontal lines, which both have a gradient of zero. We need a more curved shape,
to give us and gradient descent useful gradients. When backpropagation was introduced, the substituted a sigmoid
function (@eq-sigmoid) for the step function. However, the standard activation function is the ReLU function (@eq-relu) because of its simplicity
both mathematically and computationally.

$$
\sigma(z) = \frac{1}{1 + exp(z)}
$$ {#eq-sigmoid}

$$
ReLU(z) = max(0, z)
$$ {#eq-relu}

In both these formulae, $z$ is passed to the functions but exactly is this value?

The answer is pretty straight forward, it is the weighted sum of every connection, each with it own weight
and a bias term. Since we are dealing with multiple layers, and each layer has multiple nodes, we 
describe this weighted sum for a given node, in a given layer as $z_{j}^{l}$, where $j$, and $l$ represent 
the node and the layer, respectively. One aspect which makes the formula a bit more complex, is that the 
weighted sum is influenced by the activation of the previous layer. In the full formula, @eq-activation, we 
need to sum across all the neuron in the previous layer ($l - 1$), $m$, and multiple the weight of a given connection
by the activation of neuron it came from. In other words, if the previous layer has rendered the neuron quiet or inactive,
this should be translated for our current layer, and so on.


$$
z_{j}^{l} = \sum_{k = 1}^{m} w_{jk}^{l} a_{k}^{l - 1} + b_{j}^{l}
$$ {#eq-activation}

# Introduction to TensorFlow and Keras


In [ ]:
import tensorflow as tf

tf.config.list_physical_devices()

[^1]: This is usually the *Mean Squared Error*, but if the data has a lot of outliers, the *Mean Absolute Error* might be better. If you want 
to get fancy, you could even use *Huber Loss*, which is the combination of the too.
